# Proyecto Individual 1 
## *Machine Learning Operations (MLOps)*

### 1. Ingesta de datos

**1.1 Importamos librerías**

Importamos las librerias que vamos a usar durante el desarrollo del proyecto.

In [1]:
import pandas as pd 
import numpy as np
import json
import gzip
import ast

**1.2 Leemos los datos**

Utilizaremos 3 archivos: steam_games, user_reviews y users_items. 

In [2]:
ruta1 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/steam_games.json'
ruta2 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/user_reviews.json.gz'
ruta3 = '/Users/jpbertone/Documents/Henry Course/Labs/dataPI_MLOps/users_items.json.gz'

Mediante el uso de Pandas, leemos el archivo 'steam_games' y lo asignamos a un nombre identificatorio.

1. **steam_games**

In [3]:
df_games = pd.read_json(ruta1, lines=True)

In [4]:
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,js41637,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0


Para leer los archivos user_reviews y users_items, utilizamos otro método.

2. **user_reviews**

In [5]:
info = [] # Creamos una lista vacia donde guardaremos los registros del archivo .json

for i in gzip.open(ruta2):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_reviews = pd.DataFrame(info) 

In [6]:
df_reviews.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


3. **users_items**

In [7]:
info = [] # Creamos una lista vacia donde guardaremos los registros del archivo .json

for i in gzip.open(ruta3):
     info.append(ast.literal_eval(i.decode('utf-8')))
     
df_items = pd.DataFrame(info) 

In [8]:
df_items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


### 2. Tratamiento de datos

**2.1 Desanidar columnas**

Luego de leer los datos, se observa que los tres dataframes presentan columnas con datos anidados.
 
Para ello, vamos a utilizar las funciones .explode y .json_normalize, con el objetivo de desanidar la informacion y convertir cada clave a columna.

Luego de desanidar, concatenamos las columnas del data frame df_explode con las columnas desanidadas. 

1.  Del dataframe **df_games** no desanidamos y eliminamos la columna items, items_count, user_id y steam_id. 

    Se decide eliminar estas columnas porque ya tenemos esta infomacion en el dataset df_items_full. 


In [9]:
df_games_full = df_games.copy()
df_games_full = df_games_full.drop(columns=['items','items_count', 'user_id', 'steam_id'],axis=1)

In [12]:
df_games_full.tail(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,None,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,NaN,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None,None


2. Del dataframe **df_reviews** desanidamos la informacion que se encuentra en la columna 'reviews'.

In [13]:
df_explode = df_reviews.explode('reviews').reset_index()

reviews = pd.json_normalize(df_explode['reviews'])

df_reviews_full = pd.concat([df_explode.drop('reviews', axis=1), reviews], axis=1)

In [14]:
df_reviews_full.head(2)

,index,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.


3. Del dataframe **df_items** desanidamos la informacion que se encuentra en la columna 'items'.

In [15]:
df_explode = df_items.explode('items').reset_index()

items = pd.json_normalize(df_explode['items'])

df_items_full = pd.concat([df_explode.drop('items', axis=1), items], axis=1)

In [17]:
df_items_full.head(2)

,index,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0


**2.2 Duplicados, valores nulos e inconsistencias**

1. **df_games_full**

In [18]:
df_games_full.shape

(120445, 15)

In [19]:
df_games_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  float64
 12  id              32133 non-null  float64
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
dtypes: float64(3), object(12)
memory usage: 13.8+ MB


Vamos a observar si la columna 'precio' tiene valores numericos o otro tipo de datos que deberían ser sustituidos. 

In [20]:
df_games_full['price'].unique()

array([None, 4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, 'Free', 10.99, 1.5899999999999999, 14.99, 1.99,
       59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.3900000000000001, 'Free HITMAN™ Holiday Pack', 36.99,
       4.49, 2.0, 4.0, 9.0, 234.99, 1.9500000000000002, 1.5, 199.0, 189.0,
       6.66, 27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99, 399.0,
       20.0, 40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0,
       27.49,

Efectivamente, y como se observa en la lista de arriba, podemos observar que varios valores no son numericos y en su gran mayoría representan juegos gratuitos o demos. 

Vamos a iterar la columna con el objetivo de reemplzar esos valores por **0**. 

Ademas, observamos que existe un juego que su precio base es de $499 ('Starting at $499.0). En este caso vamos a reemplazarlo por el valor **499**

In [21]:
for i in df_games_full['price']:
    # Si dentro de la columna existe algun valor de la lista, lo reemplaza por el valor numérico 0.
    if i in ['Free to Try', 
             'Free to Play', 
             'Free To Play', 
             'Free', 'Free Demo', 
             'Play for Free!',
             'Install Now', 
             'Play WARMACHINE: Tactics Demo', 
             'Free Mod', 
             'Install Theme', 
             'Third-party', 
             'Play Now',
             'Free HITMAN™ Holiday Pack', 
             'Play the Demo', 
             'Free Movie', 
             'Free to Use']:
        df_games_full.replace(i, 0, inplace = True)
    # Si encuentra el texto, lo reemplaza por el valor numérico 499.
    elif i in ['Starting at $499.00','Starting at $449.00']:
        df_games_full.replace(i, 499, inplace = True) 



In [22]:
df_games_full['price'].unique()

array([       nan, 4.9900e+00, 0.0000e+00, 9.9000e-01, 2.9900e+00,
       3.9900e+00, 9.9900e+00, 1.8990e+01, 2.9990e+01, 1.0990e+01,
       1.5900e+00, 1.4990e+01, 1.9900e+00, 5.9990e+01, 8.9900e+00,
       6.9900e+00, 7.9900e+00, 3.9990e+01, 1.9990e+01, 7.4900e+00,
       1.2990e+01, 5.9900e+00, 2.4900e+00, 1.5990e+01, 1.2500e+00,
       2.4990e+01, 1.7990e+01, 6.1990e+01, 3.4900e+00, 1.1990e+01,
       1.3990e+01, 3.4990e+01, 7.4760e+01, 1.4900e+00, 3.2990e+01,
       9.9990e+01, 1.4950e+01, 6.9990e+01, 1.6990e+01, 7.9990e+01,
       4.9990e+01, 5.0000e+00, 4.4990e+01, 1.3980e+01, 2.9960e+01,
       1.1999e+02, 1.0999e+02, 1.4999e+02, 7.7171e+02, 2.1990e+01,
       8.9990e+01, 9.8000e-01, 1.3992e+02, 4.2900e+00, 6.4990e+01,
       5.4990e+01, 7.4990e+01, 8.9000e-01, 5.0000e-01, 2.9999e+02,
       1.2900e+00, 3.0000e+00, 1.5000e+01, 5.4900e+00, 2.3990e+01,
       4.9000e+01, 2.0990e+01, 1.0930e+01, 1.3900e+00, 3.6990e+01,
       4.4900e+00, 2.0000e+00, 4.0000e+00, 9.0000e+00, 2.3499e

Ahora podemos observar que el esos valores ya no existen y todos los valores que tenemos en la columna prices son numericos. 

El siguiente paso es convertir la columna a tipo numerico **float**.

In [92]:
df_games_full['price'] = df_games_full['price'].astype(float)

In [23]:
df_games_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  float64
 11  early_access    32135 non-null  float64
 12  id              32133 non-null  float64
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
dtypes: float64(4), object(11)
memory usage: 13.8+ MB


Como se observa en el resumen de arriba, el datasets presenta una gran cantidad de nulos.

Procedemos a eliminar los nulos siempre y cuando el registro completo tenga valores nulos en todas las columnas. Lo hacemos con la condicion 'how'.

In [24]:
df_games_full.dropna(inplace=True, how='all')

In [26]:
df_games_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  float64
 11  early_access    32135 non-null  float64
 12  id              32133 non-null  float64
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
dtypes: float64(4), object(11)
memory usage: 3.9+ MB


Podemos ver que luego de eliminar los registros vacios, el dataset pasa de 120.445 registros a 32.135. 

Los valores nulos que aun permanecen, son registros que tienen algunas columnas con valores vacios pero aun tienen informacion valiosa para el analisis. 

2. **df_reviews_full**

En el dataset de 'reviews' vamos a buscar los nulos y los vamos a eliminar.

 En este caso se toma esa decision porque la cantidad de nulos no representan un gran porcentaje del total de registros. 
 
 En total, son 28 registros los que se eliminan. 

In [74]:
print(df_reviews_full.shape)
df_reviews_full.dropna(inplace=True)
print(df_reviews_full.shape)

(59305, 10)
(59305, 10)


Además, veremos si existen duplicados. 

In [26]:
df_reviews_full.duplicated().sum()

0

1. **df_itmem_full**

Revisar nombres de juegos y unificar nombres

### 3. Feature Engineering

En el planteamiento de la problematica del proyecto, se solicita la creación de la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP. La escala a tener en cuenta es la siguiente: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo.

Para ello analizaremos la columna 'reviews' del dataframe 'df_reviews_full' obtenido en el paso 2.1

Utilizaremos la librería **TextBlob**.

In [27]:
from textblob import TextBlob

In [28]:
def calcular_sentimiento_textblob(texto):
    # Se calcula el sentimiento del texto
    analysis = TextBlob(texto)
    # El atributo sentiment.polarity devuelve la polaridad del sentimiento (entre -1 y 1)
    polaridad = analysis.sentiment.polarity
    return polaridad

def asignar_puntaje(polaridad):
    # Asigna 0 si la polaridad es negativa, 1 si es neutral y 2 si es positiva
    if polaridad < -0.1:
        return 0  # Negativo
    elif polaridad > 0.1:
        return 2  # Positivo
    else:
        return 1  # Neutral


In [29]:
# Creamos una copia del dataset a trabajar. 
df_sentimiento = df_reviews_full.copy()

# En el nuebo dataset, creamos dos columnas. Una de polaridad y otra de sentimiento. Se utlizan las funciones creadas en el paso anterior. 
df_sentimiento['polaridad'] = df_sentimiento['review'].astype(str).apply(calcular_sentimiento_textblob)
df_sentimiento['sentimiento'] = df_sentimiento['polaridad'].apply(asignar_puntaje)

In [30]:
df_sentimiento.head()

,index,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,polaridad,sentimiento
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,0.174444,2
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,0.337500,2
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,0.050000,1
3,1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,0.114583,2
4,1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,-0.036111,1


### 4. Desarrollo de funciones y disponibilización de datos (API)

**4.1 Primera función:** *def userdata( User_id : str )*

Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

Para esta funcion, vamos a crear un dataframe a partir de diferentes columnas de los distintos datasets anteriormente trabajados. 

In [100]:
df_games_full.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id
0,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,76561197970982479,7.656120e+16
1,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,js41637,7.656120e+16
2,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,evcentric,7.656120e+16
3,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,Riot-Punch,7.656120e+16
4,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,doctr,7.656120e+16


In [76]:
df_reviews_full.tail(2)

,index,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
59331,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D
59332,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 2.,,440,No ratings yet,True,so much fun :D


In [83]:
df_items_full.tail(2)

,index,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
5170013,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,519140,Minds Eyes,3.0,3.0
5170014,88309,edward_tremethick,0,76561198331598578,http://steamcommunity.com/id/edward_tremethick,NaN,NaN,NaN,NaN


In [105]:
df_games_full['id'] = df_games_full['id'].astype(float)

df_items_full['item_id'] = df_items_full['item_id'].astype(float)


In [106]:

df_items_games = df_items_full.merge(df_games_full, left_on='item_id', right_on='id', how='left')

df_items_games.drop(columns=['index'],inplace=True)

df_items_games

: 

In [34]:
def userdata(user_id):
    '''
    La siguiente función devuelve filtra por el usuario que toma la funcion como input 

    y arroja el dinero gastado por dicho usuario, el porcentaje de recomendación 

    y la cantidad de items del mismo. 

    '''

    # Calcula la suma de la columna precio filtrando por el usuario.
    money = df_games_full[df_games_full['user_id']== user_id]['price'].sum()
    # Calcula la cantidad de reviews totales de todos los usuarios.
    tot_reviews = len(df_reviews_full['user_id'].unique())
    # Calcula la cantidad de reviews filtrando por el usuario.
    tot_reviews_usuario = df_reviews_full[df_reviews_full['user_id'] == user_id]['recommend'].sum()
    # Calcula la cantidad de items filtrando por el usuario.
    cant_items = df_items_full[df_items_full['user_id'] == user_id]['items_count'].iloc[0]

    return {'Usuario:': user_id, 
            'Cantidad de dinero gastada:': money, 
            'Porcentaje de recomendación:': round((tot_reviews_usuario/tot_reviews)*100,2),
            'Cantidad de items:': cant_items
    }


In [35]:
userdata('evcentric')

{'Usuario:': 'evcentric',
 'Cantidad de dinero gastada:': 0.0,
 'Porcentaje de recomendación:': 0.02,
 'Cantidad de items:': 137}

**4.2 Segunda función:** *def countreviews( YYYY-MM-DD y YYYY-MM-DD : str )*

Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

**4.3 Tercera función:** *def genre( género : str )*

Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

**4.4 Cuarta función:** *def userforgenre( género : str )*

 Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

**4.5 Quinta función:** *def developer( desarrollador : str )*

 Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

**4.1 Sexta función:** *def sentiment_analysis( año : int )*

 Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

### 5. Deployment

### 6. Analisis exploratorio

### 7. Modelamiento